In [ ]:
npm = 0  # TODO: Isi NPM Anda

## 1. Prepare Datasets

In [ ]:
# Import packages yang relevan
import pandas as pd
import keras

from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, Dense, GlobalAveragePooling1D
from sklearn.model_selection import train_test_split

In [ ]:
# Memastikan prosesnya bersifat deterministik
keras.utils.set_random_seed(npm % 2)

In [ ]:
# Load dataset training dan testing
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

# Separasi antara teks yang digunakan dan label jenis berita
texts = train_df['description'].values  # Bisa juga teks lain seperti title
labels = train_df['news_type'].values

In [ ]:
# Jumlah kelas pada klasifikasi berdasarkan atribut news_type
num_classes = train_df['news_type'].nunique()

In [ ]:
# Melatih dan melakukan tokenisasi pada teks
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

In [ ]:
# Melakukan padding teks sepanjang maksimal maxlen
maxlen = 1000
X_train = pad_sequences(sequences, maxlen=maxlen)

In [ ]:
# Memecah data training untuk mendapatkan data validation
X_train, X_val, y_train, y_val = train_test_split(X_train, labels, test_size=0.2, random_state=42)

## 2. Train and Test Embedding Model

In [ ]:
# Mendefinisikan model dan beberapa komponen terkait
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=8))
model.add(GlobalAveragePooling1D())
model.add(Dense(num_classes, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Melatih model
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_val, y_val))

In [ ]:
# Menguji model pada data testing
test_sequences = tokenizer.texts_to_sequences(test_df['description'].values)
X_test = pad_sequences(test_sequences, maxlen=maxlen)
y_test = test_df['news_type'].values

loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy}')